In [5]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import FashionMNIST
import random
import os

# Define the hyper-parameter and load the training dataset

In [6]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


num_epochs = 5
batch_size = 32
learning_rate = 0.001

random.seed(2019)
torch.manual_seed(2019)

img_transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = FashionMNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Define a VAE architecture

In [7]:
class VAE(nn.Module):
    def __init__(self, image_channels=1, h_dim=1600, z_dim=20):
        super(VAE, self).__init__()

        self.conv1 = nn.Conv2d(image_channels, 32, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.deconv2 = nn.ConvTranspose2d(64, 32, kernel_size=5, stride=2)
        self.deconv1 = nn.ConvTranspose2d(32, image_channels, kernel_size=4, stride=2)
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        z = mu + std * eps
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h_1 = F.relu(self.conv1(x))
        h_2 = F.relu(self.conv2(h_1))
        flat = h_2.view(h_2.size(0), -1)
        z, mu, logvar = self.bottleneck(flat)
        return z, mu, logvar

    def decode(self, z):
        fc = self.fc3(z)
        reshape = fc.view(fc.size(0), 64, 5, 5)
        h_2 = F.relu(self.deconv2(reshape))
        h_1 = self.deconv1(h_2)
        return F.sigmoid(h_1)

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar


model = VAE()
if torch.cuda.is_available():
    model.cuda()

# Define a Loss function and optimizer

In [8]:
reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


# Start to training and save the reconstruction images

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 1 == 0:
        save = img.cpu().data
        save_image(img, './vae_img/input_{}.png'.format(epoch))
        save = recon_batch.cpu().data
        save_image(save, './vae_img/output_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 0 [0/60000 (0%)]	Loss: 126.765167
Train Epoch: 0 [3200/60000 (5%)]	Loss: 41.077122
Train Epoch: 0 [6400/60000 (11%)]	Loss: 37.240669
Train Epoch: 0 [9600/60000 (16%)]	Loss: 35.108440
Train Epoch: 0 [12800/60000 (21%)]	Loss: 38.540386
Train Epoch: 0 [16000/60000 (27%)]	Loss: 33.764790
Train Epoch: 0 [19200/60000 (32%)]	Loss: 34.576450
Train Epoch: 0 [22400/60000 (37%)]	Loss: 30.109039
Train Epoch: 0 [25600/60000 (43%)]	Loss: 32.457314
Train Epoch: 0 [28800/60000 (48%)]	Loss: 30.114971
Train Epoch: 0 [32000/60000 (53%)]	Loss: 30.824322
Train Epoch: 0 [35200/60000 (59%)]	Loss: 30.581726
Train Epoch: 0 [38400/60000 (64%)]	Loss: 30.100792
Train Epoch: 0 [41600/60000 (69%)]	Loss: 28.829214
Train Epoch: 0 [44800/60000 (75%)]	Loss: 31.742376
Train Epoch: 0 [48000/60000 (80%)]	Loss: 29.810665
Train Epoch: 0 [51200/60000 (85%)]	Loss: 26.980492
Train Epoch: 0 [54400/60000 (91%)]	Loss: 32.639671
Train Epoch: 0 [57600/60000 (96%)]	Loss: 25.123016
====> Epoch: 0 Average loss: 32.7933
Tr